# 01. Load and Explore Electricity Load Data

## 1. Objective
- Tải dữ liệu sơ đồ tải điện
- Hiểu cấu trúc dữ liệu (đa chuỗi thời gian)
- Kiểm tra tần suất và chất lượng dữ liệu

## 2. Data Source
- Dataset: Electricity Load Diagrams
- Source: Hugging Face
- Frequency: Hourly (expected

## 3. Data Loading
- Load dữ liệu từ file CSV / Hugging Face
- Kiểm tra kích thước dữ liệu

## 4. Data Structure Exploration
- Số lượng consumer / chuỗi thời gian
- Kiểu dữ liệu các cột
- Kiểm tra index thời gian

## 5. Data Quality Check
- Missing values
- Duplicate timestamps
- Basic statistics

## 6. Initial Observations
- Nhận xét ban đầu về dữ liệu
- Các vấn đề cần xử lý ở bước sau


## 7. Conclusion
- Tóm tắt hiểu biết về dữ liệu